In [1]:
import numpy as np
import math

In [2]:
#RGC matrices
on_parasol_responses = np.zeros((2*28,16)) #28 rows in hexagonal grid means 28 rows on a straight, but 56 rows staggered
on_parasol_positions = np.zeros((2*28,16,2)) #32 cols in hex grid means 32 staggered cols = 16 straight cols


#Bipolar matrices 512x512 area with 2 micron pitch, so 1 neuron every 2 microns, so 256x256 array
on_parasol_bipolar_responses = np.zeros((2*256,128)) #256 rows in hex grid means 256 rows on a straight, but 128 rows staggered
on_parasol_bipolar_positions = np.zeros((2*256,128,2)) #256 cols in a hex grid means 256 staggered cols = 128 straight cols


#Cone matrices 512x512 area with 2 micron pitch, same spacing as bipolars, for now we'll place all 3 cones
#in the same spot since they all feed directly into each bipolar
L_cone_responses = np.zeros((2*256,128)) #256 rows in hex grid means 256 rows on a straight, but 128 rows staggered
L_cone_positions = np.zeros((2*256,128,2)) #256 cols in a hex grid means 256 staggered cols = 128 straight cols

In [3]:
#set positions

#L_cones
#512 micron patch of retina, 256x256 hexagonal grid (flat hexagon, tops/bottoms flat, sides pointy)
#256.5*height of hexagon = 512, h ~ 1.996 microns, y value increases by 1/2 h each row
#128*(1.5*width of hexagon)+3/4*width of hexagon = 512, w = 512/192.75 ~ 2.656 microns
#set step size = 1.5*w
#odd rows go x=w/2 to x=512-(w/2+3w/4) by 1.5w, even rows go x=0+(w/2+3w/4) to x=512-w/2 by 1.5w

h=512/256.5
w=512/192.75
y_positions = np.arange(h/2, 1+512-h/2, h/2)

i = 0
while(i<np.shape(L_cone_positions)[0]): #go through all rows, use np.arrange to evenly space neurons
    #set y position for this row
    L_cone_positions[i,:,1] = np.ones(128)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        L_cone_positions[i,:,0] = np.arange(w/2+3*w/4, 1+512-w/2, 3*w/2)
    else:
        L_cone_positions[i,:,0] = np.arange(w/2, 1+512-(w/2+3*w/4), 3*w/2)
    i+=1

    

#on parasol rgc's:
#512 micron patch of retina, 28x32 hexagonal grid (flat hexagon, tops/bottoms flat, sides pointy)
#28.5*height of hexagon = 512, h ~ 17.96 microns, y value increases by 1/2 h each row
#16*(1.5*width of hexagon)+3/4*width of hexagon = 512, w = 512/24.75 ~ 20.7 microns
#set step size = 1.5*w
#odd rows go x=w/2 to x=512-(w/2+3w/4) by 1.5w, even rows go x=0+(w/2+3w/4) to x=512-w/2 by 1.5w

h=512/28.5
w=512/24.75
y_positions = np.arange(h/2, 1+512-h/2, h/2)

i = 0
while(i<np.shape(on_parasol_positions)[0]): #go through all rows, use np.arrange to evenly space neurons
    #set y position for this row
    on_parasol_positions[i,:,1] = np.ones(16)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        on_parasol_positions[i,:,0] = np.arange(w/2+3*w/4, 1+512-w/2, 3*w/2)
    else:
        on_parasol_positions[i,:,0] = np.arange(w/2, 1+512-(w/2+3*w/4), 3*w/2)
    i+=1

    
    
#on parasol bipolar cells
#512 micron patch of retina, 256x256 hexagonal grid (flat hexagon, tops/bottoms flat, sides pointy)
#256.5*height of hexagon = 512, h ~ 1.996 microns, y value increases by 1/2 h each row
#128*(1.5*width of hexagon)+3/4*width of hexagon = 512, w = 512/192.75 ~ 2.656 microns
#set step size = 1.5*w
#odd rows go x=w/2 to x=512-(w/2+3w/4) by 1.5w, even rows go x=0+(w/2+3w/4) to x=512-w/2 by 1.5w

h=512/256.5
w=512/192.75
y_positions = np.arange(h/2, 1+512-h/2, h/2)

i = 0
while(i<np.shape(on_parasol_bipolar_positions)[0]): #go through all rows, use np.arrange to evenly space neurons
    #set y position for this row
    on_parasol_bipolar_positions[i,:,1] = np.ones(128)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        on_parasol_bipolar_positions[i,:,0] = np.arange(w/2+3*w/4, 1+512-w/2, 3*w/2)
    else:
        on_parasol_bipolar_positions[i,:,0] = np.arange(w/2, 1+512-(w/2+3*w/4), 3*w/2)
    i+=1


In [4]:
# get cone responses from image

# random data:

L_cone_responses = np.random.rand(2*256,128) * 4 #lets assume up to response of 4 for now
L_cone_responses

array([[1.84004023, 1.05097116, 1.86901051, ..., 0.82417616, 2.80572345,
        1.91177209],
       [1.10335511, 3.30766174, 1.54624042, ..., 0.45998965, 1.7629277 ,
        3.07157191],
       [0.37577007, 1.53651911, 3.40512341, ..., 3.47550684, 3.17129492,
        2.86826276],
       ...,
       [1.32099956, 1.21537676, 3.63095803, ..., 3.77957171, 0.23942132,
        3.79467795],
       [3.75639111, 2.74372805, 2.98924455, ..., 2.17972236, 2.17977063,
        1.3271769 ],
       [2.59571636, 1.14419663, 3.84519096, ..., 2.29838271, 3.3591986 ,
        3.92938442]])

In [5]:
# get response of bipolar cells from electrodes


#spacial response

#loop through all cones, find "bounding box",
#go through all bipolars and add response to that electrode if in box otherwise go to next neuron to speed up simulation

br = 0


i=0
j=0
while(i<np.shape(L_cone_positions)[0]):
    while(j<np.shape(L_cone_positions)[1]):
        x_cone = L_cone_positions[i,j,0]
        y_cone = L_cone_positions[i,j,1]
        #should be able to end simulation at 10 microns away
        x1 = x_cone - 12
        x2 = x_cone + 12
        y1 = y_cone - 12
        y2 = y_cone + 12
        
        cone_input = L_cone_responses[i,j]
        
        #if(x1<0):
        #    x1 = 0
        #if(y1<0):
        #    y1 = 0
        
        #~start x = ratio of x1 in total x simulation space * # x positions in matrix - spacing
        #start_x = math.floor((x1/512.0)*np.shape(on_parasol_bipolar_positions)[0])
        #if(start_x>4):
        #    start_x-=4
        
        #start_y = math.floor((y1/512.0)*np.shape(on_parasol_bipolar_positions)[1])
        #if(start_y>4):
        #    start_y-=4
        
        #print(start_x)
        #print(start_y)
        #print("---")
        #go through all neurons and simulate responses
        
        #on parasol bipolar simulation:
        a=0
        b=0
        while(a<np.shape(on_parasol_bipolar_positions)[0]):
            while(b<np.shape(on_parasol_bipolar_positions)[1]):
                x=on_parasol_bipolar_positions[a,b,0]
                y=on_parasol_bipolar_positions[a,b,1]
                if(y>=y2):
                    br = 1
                    break
                if(x>=x2):
                    break
                if(x1<x and x<x2 and y1<y and y<y2):
                    #add to response since it's inside the range of the bipolar
                    
                    #Lecture 6: G(x,y) = 1/(2*pi*sigma^2) * e^(-(r^2)/(2*sigma^2))
                    #gaussian center std dev values are 4 for parasol an 2 for midget
                    #gaussian surround std dev values from golden are 9 * center std dev
                    #center peak has value 1, surround peak has value 0.01 for all
                    #(all from the end of page 3)

                    #gaussian with amplitude included: G(x,y) = a * e^(-(r^2)/(2*sigma^2))

                    #diff of gaussians with amplitude: DoG(x,y) = a_cent * e^(-(r^2)/(2*sig_cent^2))
                    # - a_sur * e^(-(r^2)/(2*sig_sur^2))

                    #r(x,y) = sqrt((x-x_elec)^2 + (y-y_elec)^2)
                    
                    r = math.sqrt((x - x_cone)**2 + (y-y_cone)**2)
                    a_cent = 1
                    a_sur = 0.01
                    sig_cent = 4
                    sig_sur = sig_cent * 9
                    DoG = a_cent * math.e**(-(r**2)/(2*sig_cent**2)) - a_sur * math.e**(-(r**2)/(2*sig_sur**2))
                    
                    on_parasol_bipolar_responses[a,b] += DoG * cone_input
                
                #otherwise just continue to next neuron
                
                b+=1
            
            if(br==1):
                br = 0
                break
            b=0
            a+=1
                    
        j+=1
    j=0
    i+=1




In [6]:
on_parasol_bipolar_responses[0]

array([18.54281535, 25.90772661, 27.05640702, 26.03800057, 26.71212599,
       27.17706596, 28.1066685 , 29.5354053 , 28.99669303, 27.29798515,
       26.18867467, 25.95383385, 26.19047406, 25.18760784, 24.94602504,
       26.53647817, 28.26910345, 29.56013407, 29.47788693, 25.56837127,
       22.41172737, 23.01317689, 22.70696846, 21.277581  , 22.21197375,
       24.9917717 , 25.16005149, 21.66544567, 19.62341894, 20.38638163,
       21.52801148, 23.64929818, 26.88250298, 28.95160011, 29.98701836,
       29.77207577, 26.32445959, 22.11237812, 22.04867337, 25.4136844 ,
       29.81809424, 31.75407235, 28.21682879, 22.91336388, 20.43357245,
       21.06985429, 25.45689711, 27.40679964, 25.06687732, 23.4078268 ,
       23.16133002, 25.14901572, 27.28368109, 26.57734685, 25.73559126,
       24.1539245 , 22.804331  , 25.56001225, 29.78460209, 30.17835455,
       26.52135933, 23.63205318, 22.39412685, 21.46146162, 24.66915435,
       28.17837239, 27.83936338, 26.41765488, 26.71655481, 28.80

In [7]:

#temporal response


#Since golden paper didn't give any information on the temporal filter (and we are using slightly different
#temporal modeling with single frame of picture shown then nothing for the rest of the 0.4 s), we will use
#exponential decay to model the temporal behavior with the initial value starting at the spacial response
#and the decay value to be 50 (0.02s half life of neuron response strength, close to curve of golden
#temporal response)

#response = init_response * e^(decay_val*t)

decay_val = -50


#on parasol temporal:

initial_on_parasol_bipolar_responses = np.copy(on_parasol_bipolar_responses)

t=0.001 #time steps of 0.001s (400 time periods)
while t<0.4: #simulate 0.4s like golden paper did
    i=0
    j=0
    while(i<np.shape(on_parasol_bipolar_positions)[0]):
        while(j<np.shape(on_parasol_bipolar_positions)[1]):
            #calculate current time's temporal response value with exponential decay equation, then add it
            #to the overall bipolar response
            
            init_response = initial_on_parasol_bipolar_responses[i,j]
            
            response = init_response * math.e**(decay_val*t)
            
            on_parasol_bipolar_responses[i,j] += response #sum over temporal response for each neuron
            
            j+=1
        i+=1
    
    t+=0.001


In [8]:
#simulate rgc responses to bipolars

#loop through all bipolars, find "bounding box" (within ~30 microns of electrode in each direction),
#go through all bipolars and add response to that electrode if in box otherwise go to next neuron to speed up simulation

br = 0

#on parasol rgc:

i=0
j=0
while(i<np.shape(on_parasol_bipolar_positions)[0]):
    while(j<np.shape(on_parasol_bipolar_positions)[1]):
        x_elec = on_parasol_bipolar_positions[i,j,0]
        y_elec = on_parasol_bipolar_positions[i,j,1]
        x1 = x_elec - 25 #past 25 microns away is .6% & 1.5% of normal curves (center and surround)
        x2 = x_elec + 25
        y1 = y_elec - 25
        y2 = y_elec + 25
        
        response = on_parasol_bipolar_responses[i,j]
        
        #go through all rgc and simulate responses
        
        #on parasol rgc simulation:
        a=0
        b=0
        while(a<np.shape(on_parasol_positions)[0]):
            while(b<np.shape(on_parasol_positions)[1]):
                x=on_parasol_positions[a,b,0]
                y=on_parasol_positions[a,b,1]
                if(y>=y2):
                    br = 1
                    break
                if(x>=x2):
                    break
                if(x1<x and x<x2 and y1<y and y<y2):
                    #add to response since it's inside the range of the bipolar
                    
                    #Lecture 6: G(x,y) = 1/(2*pi*sigma^2) * e^(-(r^2)/(2*sigma^2))
                    #gaussian center std dev values are 10, 8, 4, and 3.5 microns (on par, off par, on mid, off mid)
                    #gaussian surround std dev values from golden are 1.15 * center std dev
                    #center peak has value 1, surround peak has value 0.375 for parasol and 0.5 for midget
                    #(all from the end of page 3 and beginning of page 4)

                    #gaussian with amplitude included: G(x,y) = a * e^(-(r^2)/(2*sigma^2))

                    #diff of gaussians with amplitude: DoG(x,y) = a_cent * e^(-(r^2)/(2*sig_cent^2))
                    # - a_sur * e^(-(r^2)/(2*sig_sur^2))

                    #r(x,y) = sqrt((x-x_elec)^2 + (y-y_elec)^2)

                    r = math.sqrt((x - x_elec)**2 + (y-y_elec)**2)
                    a_cent = 1
                    a_sur = 0.375
                    sig_cent = 10
                    sig_sur = sig_cent * 1.15
                    DoG = a_cent * math.e**(-(r**2)/(2*sig_cent**2)) - a_sur * math.e**(-(r**2)/(2*sig_sur**2))
                    on_parasol_responses[a,b] += DoG * response
                    
                #otherwise just continue to next neuron
                
                b+=1
            
            if(br==1):
                br = 0
                break
            b=0
            a+=1
        
        j+=1
    j=0
    i+=1



In [9]:
#convert response to firing rate


#go through all rgc responses and convert response to firing rate

#-----------------------------------------------------------------------------------------------
#important: adjust the exponential function later once voltages are adjusted

#we'll use an exponential function to get spike rate from rgc response (golden got this from pillow et al 2008)
#large rgc response is 0.1, so exponential goes from e^0=1 to e^0.1=1.1
#6 Hz seems to be a large neuron firing rate for an RGC (from other studies), so we'll use this as max_rate
#exponential function: firing_rate = max_rate*10*(e^(response)-1)
#subtract 1 from exponential so we have values starting from 0, multiply by 10 to get our max firing rate
#to be about equal to our inputted max firing rate

max_rate = 6 #6Hz seems to be a very large RGC firing rate

#hopefully the learned reconstruction matrix will be able to compensate for any differences we've made in
#our equations (as long as we use the same equations here and in learning)


#on parasol rgcs:

i=0
j=0
while(i<np.shape(on_parasol_positions)[0]):
    while(j<np.shape(on_parasol_positions)[1]):
        response = on_parasol_responses[i,j]
        
        firing_rate = max_rate*10*(math.e**(response)-1)
        
        if(firing_rate < 0):
            firing_rate = 0
        
        on_parasol_responses[i,j] = firing_rate
        
        j+=1
    j=0
    i+=1



C:\Users\rhyst\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: overflow encountered in double_scalars


In [10]:
on_parasol_bipolar_responses[0]

array([380.2049723 , 531.21633888, 554.7690732 , 533.88749781,
       547.70987764, 557.24308414, 576.30380932, 605.59886648,
       594.5530204 , 559.72243127, 536.97694449, 532.16172935,
       537.01383958, 516.45090365, 511.49744967, 544.10836548,
       579.63440252, 606.10590937, 604.41950032, 524.25814048,
       459.53378854, 471.86600943, 465.58746092, 436.27906239,
       455.43800703, 512.43544692, 515.88588365, 444.23190427,
       402.36184829, 418.00576227, 441.41393069, 484.90914634,
       551.20331576, 593.62842776, 614.85881587, 610.45159723,
       539.7611011 , 453.39588165, 452.08966881, 521.0864151 ,
       611.39516749, 651.09078489, 578.56255432, 469.81942701,
       418.97337063, 432.01979936, 521.97245574, 561.95358167,
       513.97542496, 479.95797703, 474.90376601, 515.65960437,
       559.42913856, 544.94634363, 527.68684691, 495.25608832,
       467.58379867, 524.08674562, 610.70843889, 618.78200496,
       543.79836623, 484.55555205, 459.17290444, 440.04